In [14]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [39]:
import addict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
from tqdm.notebook import tqdm

import celeri
celeri = reload(celeri)

# Plotting the global model is much much faster with tex fonts turned off
plt.rcParams['text.usetex'] = False

In [40]:
command_file_name = './data/western_north_america/basic_command.json'
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure, block = celeri.assign_block_labels(segment, station, block, mogi, sar)
celeri.plot_block_labels(segment, block, station, closure)
block

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

    other1  other2  other3  other4  other5  other6            name  \
0        0       0       0       0       0       0  NorthAmerica     
1        0       0       0       0       0       0  ele56            
2        0       0       0       0       0       0  newblock15       
3        0       0       0       0       0       0  newblock03       
4        0       0       0       0       0       0  newblock28       
5        0       0       0       0       0       0  newblock31       
6        0       0       0       0       0       0  Restoftheworld   
7        0       0       0       0       0       0  ele37            
8        0       0       0       0       0       0  ele67            
9        0       0       0       0       0       0  ele40            
10       0       0       0       0       0       0  ele60            
11       0       0       0       0       0       0  ele44            
12       0       0       0       0       0       0  ele35            
13       0       0  

In [32]:
segment

,name,lon1,lat1,lon2,lat2,dip,res,other3,other6,other7,...,mid_x,mid_y,mid_z,centroid_x,centroid_y,centroid_z,centroid_lon,centroid_lat,west_labels,east_labels
0,BRa ...,247.087,43.873,247.764,42.286,90.0,100,0,0,0,...,-1.785035e+06,-4.294581e+06,4.349089e+06,-1.785035e+06,-4.294581e+06,7.5,247.429868,43.080056,1,0
1,BRb ...,245.927,44.722,247.087,43.873,90.0,100,0,0,0,...,-1.816369e+06,-4.179592e+06,4.447040e+06,-1.816369e+06,-4.179592e+06,7.5,246.511179,44.298988,1,0
2,Baja_connect_eleaa ...,244.321,31.193,244.760,31.084,90.0,100,0,0,0,...,-2.342802e+06,-4.920762e+06,3.292678e+06,-2.342802e+06,-4.920762e+06,7.5,244.540626,31.138687,3,2
3,Baja_connect_eleab ...,244.760,31.084,245.238,31.030,90.0,100,0,0,0,...,-2.305331e+06,-4.943588e+06,3.284926e+06,-2.305331e+06,-4.943588e+06,7.5,244.999067,31.057221,3,2
4,Baja_connect_eleba ...,245.238,31.030,245.703,30.976,90.0,100,0,0,0,...,-2.265856e+06,-4.965207e+06,3.279782e+06,-2.265856e+06,-4.965207e+06,7.5,245.470566,31.003209,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
832,wcnsbbab ...,242.158,42.335,242.190,41.761,90.0,100,0,0,0,...,-2.207137e+06,-4.181621e+06,4.264616e+06,-2.207137e+06,-4.181621e+06,7.5,242.174072,42.048008,30,1
833,wcnsbbba ...,242.158,42.335,242.567,42.567,90.0,100,0,0,0,...,-2.179426e+06,-4.162154e+06,4.297782e+06,-2.179426e+06,-4.162154e+06,7.5,242.362123,42.451184,30,1
834,wcnsbbbba ...,242.567,42.567,243.233,42.649,90.0,100,0,0,0,...,-2.134890e+06,-4.171902e+06,4.310664e+06,-2.134890e+06,-4.171902e+06,7.5,242.899782,42.608484,10,1
835,wcnsbbbbb ...,243.233,42.649,243.899,42.731,90.0,100,0,0,0,...,-2.083505e+06,-4.190907e+06,4.317367e+06,-2.083505e+06,-4.190907e+06,7.5,243.565781,42.690484,10,1


In [ ]:
assembly = addict.Dict()
operators = addict.Dict()
assembly = celeri.merge_geodetic_data(assembly, station, sar)
assembly, operators.block_motion_constraints = celeri.block_constraints(assembly, block, command)
assembly, operators.slip_rate_constraints = celeri.slip_rate_constraints(assembly, segment, block, command)
operators.okada_segment_station = celeri.get_segment_station_operator_okada(segment, station, command)
operators.slip_rate_segment_block = celeri.get_fault_slip_rate_partials(segment, block)
assembly.index.sz_elastic = operators.okada_segment_station.shape # Not sure this is correct
assembly.index.sz_slip = operators.slip_rate_segment_block.shape # Not sure this is correct
operators.block_rotation = celeri.get_block_rotation_operator(station, block)
assembly.index.sz_rotation = operators.block_rotation.shape # Not sure this is correct
assembly = celeri.station_row_keep(assembly) # Not sure this is correct


In [ ]:
celeri.plot_segment_displacements(segment, station, command, segment_idx=0, strike_slip=1, dip_slip=0, tensile_slip=0, lon_min=235, lon_max=255, lat_min=30, lat_max=50, quiver_scale=1e-1)

In [ ]:
model = addict.Dict()

# function [strain, Index, Model] def get_strain_operator(block, station, segment, command, assembly)
"""
Wrapper function for calculating strain partial derivatives.
"""

# [strain, Index.strainBlock]                = deal(zeros(3*numel(Station.lon), 0), []);
# [Model.lonStrain, Model.latStrain]         = deal(zeros(size(Block.interiorLon)));

# TODO: rename block_rotation_flag
# TODO: block.strain_flag for this: Add to matlab converter
# if any(block.strain_flag) # If any blocks have strain turned on
#    if command.strain_method == 1:
#          [strain, Index.strainBlock,...
#           Model.lonStrain, Model.latStrain]   = GetStrainCentroidPartials(Block, Station, Segment);
#    if command.strain_method == 2: # Solve for the reference coordinates
#          [strain, Index.strainBlock]          = GetStrain56Partials(Block, Station, Segment);
#    if command.strain_method == 3: # Solve for the reference latitude only
#          [strain, Index.strainBlock]          = GetStrain34Partials(Block, Station, Segment);
#    if command.strain_method == 4: # Use the simulated annealing approach to solve for reference coordinates inside the block boundaries
#          [strain, Index.strainBlock]          = GetStrainSearchPartials(Block, Station, Segment);
#    end      
# end

# function [G, strainBlockIdx, cLon, cLat, toDelete]  = GetStrainCentroidPartials(Block, Station, Segment)
# % Calculate strain partial derivatives
# nStations                                   = numel(Station.lon);
# nBlocks	                                   = numel(Block.interiorLon);
# G                                           = zeros(3*nStations, 3*nBlocks);
# R              				                 = 6371*1e6; % radius of Earth in mm
# minSta                                      = 3; % 3 for Jack

# % Convert station positions into radians and co-latitude
# Station.lon                                 = deg2rad(Station.lon);
# Station.lat(find(Station.lat)>=0)           = 90 - Station.lat(find(Station.lat)>=0);
# Station.lat(find(Station.lat)<0)            = -90 - Station.lat(find(Station.lat)<0);
# Station.lat                                 = deg2rad(Station.lat);

# % Create array containing column index limits of the design matrix
# firstCol                                    = 3*Station.blockLabel - 2;
# lastCol                                     = 3*Station.blockLabel;

# [cLon, cLat]									 	  = deal(zeros(nBlocks, 1));	

# for iStation = 1:nStations
#    if Block.rotationInfo(Station.blockLabel(iStation)) == 1
#       % Partials with block centroid
#       % The block "centroid" is only an approximation given by the mean of its coordinates
#       % This should work reasonably well for the "chopped" case but is not exact or general
#       idx1                                  = find(Segment.eastLabel == Station.blockLabel(iStation));
#       idx2                                  = find(Segment.westLabel == Station.blockLabel(iStation));
#       idx                                   = union(idx1, idx2);
#       lonVec                                = [Segment.lon1(idx);Segment.lon2(idx)];
#       latVec                                = [Segment.lat1(idx);Segment.lat2(idx)];
# %       [lat0 lon0]                           = meanm(latVec(:), lonVec(:));
#       lat0                                  = mean(latVec(:));
#       lon0                                  = mean(lonVec(:));
#       cLon(Station.blockLabel(iStation))	  = lon0;
#       cLat(Station.blockLabel(iStation))	  = lat0;
#       lon0                                  = deg2rad(lon0);
#       lat0(lat0>=0)                         = 90 - lat0(lat0>=0);
#       lat0(lat0<0)                          = -90 - lat0(lat0<0);      
#       lat0                                  = deg2rad(lat0);
#       G(3*iStation-2:3*iStation, firstCol(iStation):lastCol(iStation)) = [R*(Station.lon(iStation)-lon0).*sin(lat0) R*(Station.lat(iStation)-lat0) 0;...
#                                                                           0 R*(Station.lon(iStation)-lon0).*sin(lat0) R*(Station.lat(iStation)-lat0);...
#                                                                           0 0 0];
#    end
# end

# % Get rid of blocks that don't have enough stations on them
# for i = 1:size(G, 2)/3;
#    j                                        = 3*i-1;
#    numSta(i)                                = numel(find(G(:, j)));
# end
# toDelete                                    = [3*find(numSta<2*minSta)-2 3*find(numSta<2*minSta)-1 3*find(numSta<2*minSta)];
# strainBlockIdx                              = setdiff(1:1:nBlocks*3, toDelete);
# G(:,toDelete)                               = [];





In [ ]:
block

In [ ]:
# import copy
# b = copy.deepcopy(block)
# block.block_label

# b = b.set_index(block.block_label, append=True).sort_index(level=1).reset_index(1, drop=True)
# b = b.reset_index()
# b.head()

In [ ]:
block